# 下载并存储数据

本笔记本包含有关下载 Quandl Wiki 股票价格的信息以及我们在整本书中使用的其他一些资源。

## 导入和设置

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

## 设置数据存储路径

如果您想将数据存储在其他地方并相应地更改笔记本，请修改路径

In [93]:
DATA_STORE = Path('assets.h5')

## Quandl 维基价格

> Quandl 已于 2018 年底 [被纳斯达克收购](https://www.nasdaq.com/about/press-center/nasdaq-acquires-quandl-advance-use-alternative-data)。2021 年，纳斯达克 [整合Quandl 的数据平台](https://data.nasdaq.com/)。免费的美国股票数据仍可通过 [新 URL](https://data.nasdaq.com/databases/WIKIP/documentation) 获得，但受下述限制。

[纳斯达克](https://data.nasdaq.com/) 提供包含股票价格、股息和拆分的[数据集](/home/stefan/drive/machine-learning-for-trading/data/create_datasets.ipynb) 3000 家美国上市公司。在被收购之前（2018 年 4 月 11 日），Quandl 宣布终止社区支持（更新）。历史数据可作为展示机器学习解决方案应用的第一步，只需确保您使用当前的专业级数据设计和测试自己的算法即可。

1. 按照说明创建一个免费的[纳斯达克账户](https://data.nasdaq.com/sign-up)
2. [下载](https://data.nasdaq.com/tables/WIKIP/WIKI-PRICES/export)整个WIKI/PRICES数据
3. 提取 .zip 文件，
4. 移至此目录并重命名为 wiki_prices.csv
5. 运行以下代码以快速 HDF 格式存储（详见[第 02 章市场和基础数据](../02_market_and_fundamental_data)）。

In [8]:
df = (pd.read_csv('wiki_prices.csv',
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())

print(df.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1962-01-02 00:00:00'), 'ARNC') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 12 columns):
 #   Column       Non-Null Count     Dtype  
---  ------       --------------     -----  
 0   open         15388776 non-null  float64
 1   high         15389259 non-null  float64
 2   low          15389259 non-null  float64
 3   close        15389313 non-null  float64
 4   volume       15389314 non-null  float64
 5   ex-dividend  15389314 non-null  float64
 6   split_ratio  15389313 non-null  float64
 7   adj_open     15388776 non-null  float64
 8   adj_high     15389259 non-null  float64
 9   adj_low      15389259 non-null  float64
 10  adj_close    15389313 non-null  float64
 11  adj_volume   15389314 non-null  float64
dtypes: float64(12)
memory usage: 1.4+ GB
None


### Wiki 价格元数据

> QUANDL 用于在其网站上提供一些股票元数据；我正在提供该文件以允许读者运行本书中的一些示例：

不要使用 QUANDL API，而是按照描述加载文件“wiki_stocks.csv”并以 HDF5 格式存储。

In [5]:
df = pd.read_csv('wiki_stocks.csv')
# no longer needed
# df = pd.concat([df.loc[:, 'code'].str.strip(),
#                 df.loc[:, 'name'].str.split('(', expand=True)[0].str.strip().to_frame('name')], axis=1)

print(df.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/stocks', df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    3199 non-null   object
 1   name    3199 non-null   object
dtypes: object(2)
memory usage: 50.1+ KB
None


## 标准普尔 500 指数价格

以下代码从 FRED 下载标准普尔 500 指数的历史价格（仅免费提供最近 10 年的每日数据）

In [6]:
df = web.DataReader(name='SP500', data_source='fred', start=2009).squeeze().to_frame('close')
print(df.info())
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/fred', df)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2609 entries, 2010-06-16 to 2020-06-15
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   2517 non-null   float64
dtypes: float64(1)
memory usage: 40.8 KB
None


或者，从 [stooq.com](https://stooq.com/q/?s=%5Espx&c=1d&t=l&a=lg&b=0) 下载 S&P500 数据；在撰写本文时，该数据自 1789 年以来可用。您可以在右下角从波兰语切换为英语。

我们存储 1950-2020 年的数据：

In [9]:
sp500_stooq = (pd.read_csv('^spx_d.csv', index_col=0,
                     parse_dates=True).loc['1950':'2019'].rename(columns=str.lower))
print(sp500_stooq.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17700 entries, 1950-01-03 to 2019-12-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    17700 non-null  float64
 1   high    17700 non-null  float64
 2   low     17700 non-null  float64
 3   close   17700 non-null  float64
 4   volume  17700 non-null  float64
dtypes: float64(5)
memory usage: 829.7 KB
None


In [10]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stooq', sp500_stooq)

### 标准普尔 500 成分股

以下代码从 [维基百科](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies) 下载当前标准普尔 500 成分股。

In [13]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url, header=0)[0]

In [14]:
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [15]:
df.columns = ['ticker', 'name', 'sec_filings', 'gics_sector', 'gics_sub_industry',
              'location', 'first_added', 'cik', 'founded']
df = df.drop('sec_filings', axis=1).set_index('ticker')

In [16]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 505 entries, MMM to ZTS
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               505 non-null    object
 1   gics_sector        505 non-null    object
 2   gics_sub_industry  505 non-null    object
 3   location           505 non-null    object
 4   first_added        408 non-null    object
 5   cik                505 non-null    int64 
 6   founded            234 non-null    object
dtypes: int64(1), object(6)
memory usage: 31.6+ KB
None


In [17]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('sp500/stocks', df)

## 美国上市公司的元数据

以下下载在纳斯达克、美国证券交易所和纽约证券交易所交易的[公司](https://www.nasdaq.com/screening/companies-by-name.aspx) 的几个属性

> 更新：不幸的是，纳斯达克已禁用自动下载。但是，您仍然可以在填写交易所名称时访问以下网址并手动下载文件。因此对于 AMEX，URL 变为“https://www.nasdaq.com/market-activity/stocks/screener?exchange=AMEX&letter=0&render=download”。
>

In [12]:
# no longer works!
url = 'https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange={}&render=download'
exchanges = ['NASDAQ', 'AMEX', 'NYSE']
df = pd.concat([pd.read_csv(url.format(ex)) for ex in exchanges]).dropna(how='all', axis=1)
df = df.rename(columns=str.lower).set_index('symbol').drop('summary quote', axis=1)
df = df[~df.index.duplicated()]
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
Index: 6988 entries, TXG to ZYME
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       6988 non-null   object 
 1   lastsale   6815 non-null   float64
 2   marketcap  5383 non-null   object 
 3   ipoyear    3228 non-null   float64
 4   sector     5323 non-null   object 
 5   industry   5323 non-null   object 
dtypes: float64(2), object(4)
memory usage: 382.2+ KB
None


In [13]:
df.head()

,name,lastsale,marketcap,ipoyear,sector,industry
symbol,,,,,,
TXG,"10x Genomics, Inc.",88.4200,$8.7B,2019.0,Capital Goods,Biotechnology: Laboratory Analytical Instruments
YI,"111, Inc.",6.6200,$545.22M,2018.0,Health Care,Medical/Nursing Services
PIH,"1347 Property Insurance Holdings, Inc.",4.5443,$27.58M,2014.0,Finance,Property-Casualty Insurers
PIHPP,"1347 Property Insurance Holdings, Inc.",25.4202,NaN,NaN,Finance,Property-Casualty Insurers
TURN,180 Degree Capital Corp.,1.8300,$56.95M,NaN,Finance,Finance/Investors Services


### 将市值信息转换为数字格式

市值以字符串形式提供，因此我们需要将其转换为数字格式。

In [14]:
mcap = df[['marketcap']].dropna()
mcap['suffix'] = mcap.marketcap.str[-1]
mcap.suffix.value_counts()

M    3148
B    2235
Name: suffix, dtype: int64

仅保留具有值单位的值：

In [15]:
mcap = mcap[mcap.suffix.str.endswith(('B', 'M'))]
mcap.marketcap = pd.to_numeric(mcap.marketcap.str[1:-1])
mcaps = {'M': 1e6, 'B': 1e9}
for symbol, factor in mcaps.items():
    mcap.loc[mcap.suffix == symbol, 'marketcap'] *= factor
mcap.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5383 entries, TXG to ZYME
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   marketcap  5383 non-null   float64
 1   suffix     5383 non-null   object 
dtypes: float64(1), object(1)
memory usage: 286.2+ KB


In [16]:
df['marketcap'] = mcap.marketcap
df.marketcap.describe(percentiles=np.arange(.1, 1, .1).round(1)).apply(lambda x: f'{int(x):,d}')

count                5,383
mean         8,058,312,556
std         46,063,490,648
min              1,680,000
10%             41,436,000
20%            104,184,000
30%            192,888,000
40%            335,156,000
50%            587,760,000
60%          1,120,000,000
70%          2,140,000,000
80%          4,480,000,000
90%         13,602,000,000
max      1,486,630,000,000
Name: marketcap, dtype: object

### 存储结果


文件 us_equities_meta_data.csv 包含用于许多示例的数据版本。加载使用
```
df = pd.read_csv('us_equities_meta_data.csv')
```
并继续以 HDF5 格式存储

In [5]:
df = pd.read_csv('us_equities_meta_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6834 entries, 0 to 6833
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     6834 non-null   object 
 1   name       6834 non-null   object 
 2   lastsale   6718 non-null   float64
 3   marketcap  5766 non-null   float64
 4   ipoyear    3038 non-null   float64
 5   sector     5288 non-null   object 
 6   industry   5288 non-null   object 
dtypes: float64(3), object(4)
memory usage: 373.9+ KB


In [7]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('us_equities/stocks', df.set_index('ticker'))

## MNIST 数据

In [36]:
mnist = fetch_openml('mnist_784', version=1)

In [37]:
print(mnist.DESCR)

**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image b

In [38]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [39]:
mnist_path = Path('mnist')
if not mnist_path.exists():
    mnist_path.mkdir()

In [40]:
np.save(mnist_path / 'data', mnist.data.astype(np.uint8))
np.save(mnist_path / 'labels', mnist.target.astype(np.uint8))

## 时尚 MNIST 图像数据

我们将使用 [Zalando Research](https://github.com/zalandoresearch/fashion-mnist) 创建的 Fashion MNIST 图像数据进行一些演示。

In [12]:
fashion_mnist = fetch_openml(name='Fashion-MNIST')

In [13]:
print(fashion_mnist.DESCR)

**Author**: Han Xiao, Kashif Rasul, Roland Vollgraf  
**Source**: [Zalando Research](https://github.com/zalandoresearch/fashion-mnist)  
**Please cite**: Han Xiao and Kashif Rasul and Roland Vollgraf, Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms, arXiv, cs.LG/1708.07747  

Fashion-MNIST is a dataset of Zalando's article images, consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Fashion-MNIST is intended to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits. 

Raw data available at: https://github.com/zalandoresearch/fashion-mnist

### Target classes
Each training and test example is assigned to one of the following labels:
Label  Description  
0  T-shirt/top  
1  Trouser  
2  Pullover  
3  Dress  
4  

In [33]:
label_dict = {0: 'T-shirt/top',
              1: 'Trouser',
              2: 'Pullover',
              3: 'Dress',
              4: 'Coat',
              5: 'Sandal',
              6: 'Shirt',
              7: 'Sneaker',
              8: 'Bag',
              9: 'Ankle boot'}

In [34]:
fashion_path = Path('fashion_mnist')
if not fashion_path.exists():
    fashion_path.mkdir()

In [35]:
pd.Series(label_dict).to_csv(fashion_path / 'label_dict.csv', index=False, header=None)

In [31]:
np.save(fashion_path / 'data', fashion_mnist.data.astype(np.uint8))
np.save(fashion_path / 'labels', fashion_mnist.target.astype(np.uint8))


## 债券价格指数

以下代码从美联储经济数据服务 ([FRED](https://fred.stlouisfed.org/)) 下载几个债券指数


> 警告：不幸的是，大部分数据已被 [最近删除](https://news.research.stlouisfed.org/2022/01/ice-benchmark-administration-ltd-iba-data-to-be-removed- from-fred/) 来自 FRED 服务。对于书中的例子来说并不重要，所以你可以忽略它。

In [27]:
securities = {'BAMLCC0A0CMTRIV'   : 'US Corp Master TRI',
              'BAMLHYH0A0HYM2TRIV': 'US High Yield TRI',
              'BAMLEMCBPITRIV'    : 'Emerging Markets Corporate Plus TRI',
              'GOLDAMGBD228NLBM'  : 'Gold (London, USD)',
              'DGS10'             : '10-Year Treasury CMR',
              }

df = web.DataReader(name=list(securities.keys()), data_source='fred', start=2000)
df = df.rename(columns=securities).dropna(how='all').resample('B').mean()

with pd.HDFStore(DATA_STORE) as store:
    store.put('fred/assets', df)